# Audio key estimation of digital music with CNNs

## Data Preprocessing

### Million Song Dataset
- utilized to select appropriate song samples
- holds information about key and mode per song (used as targets)

Juypter Notebook <a href='./00.hlp/msd/msd.ipynb'>msd</a>

outputs: csv file *songs_conf=75_tracks_filt.csv*, which holds all songs with key confidence and mode confidence > 0.75

In [2]:
# LIST SELECTED SONGS
import os
import pandas as pd
from IPython.display import display

selsongsfile = os.path.join ('00.hlp', 'msd', 'songs_conf=75_tracks_filt.csv')
selsongs = pd.read_csv (selsongsfile, header=0, index_col=0)
display (selsongs.head (1))
print ('[i] number of records:', len (selsongs))

,key,key_confidence,mode,mode_confidence,track_id,song_id,artist_name,song_title
0,7,0.896,1,0.852,TRMMMGL128F92FD6AB,SOHSSPG12A8C144BE0,Clifford T. Ward,Mad About You


[i] number of records: 47913


In [12]:
# LOAD AUDIO DATASET
import os
import numpy as np
from sklearn import datasets

PARAM_RND_STATE = 42

container_path = os.path.join ('src_audio')
load_content = False
description = ['key C, mode minor', 'key C, mode major',
               'key C#, mode minor', 'key C#, mode major',
               'key D, mode minor', 'key D, mode major',
               'key D#, mode minor', 'key D#, mode major',
               'key E, mode minor', 'key E, mode major',
               'key F, mode minor', 'key F, mode major',
               'key F#, mode minor', 'key F#, mode major',
               'key G, mode minor', 'key G, mode major',
               'key G#, mode minor', 'key G#, mode major',
               'key A, mode minor', 'key A, mode major',
               'key A#, mode minor', 'key A#, mode major',
               'key B, mode minor', 'key B, mode major']

src_audio_data = datasets.load_files (container_path=container_path,
                                      description=description,
                                      load_content=load_content,
                                      random_state=PARAM_RND_STATE)

In [14]:
# FYI: LIST SOME OF THE USED SONGS
filenames = list (os.path.basename (filepath) for filepath in src_audio_data['filenames'])
usedsongs_track_id = list (os.path.splitext (fn)[0] for fn in filenames)
usedsongs = selsongs.query ('track_id in @usedsongs_track_id')

display (usedsongs.sample(5))
print ('[i] min of: key_confidence =', usedsongs['key_confidence'].min (), ',', \
       'mode_confidence =', usedsongs['mode_confidence'].min ())
print ('[i] number of records:', len (usedsongs))

,key,key_confidence,mode,mode_confidence,track_id,song_id,artist_name,song_title
30919,2,0.958,0,0.939,TRLSEBW12903CAC63F,SOXMBON12AB018C9FB,Greg Lake,C'Est La Vie
15891,10,1.000,0,0.927,TRQEAPQ128F92F9A58,SOVCOJV12A8C1435DA,Optical,Can U Read?
38767,4,1.000,1,0.989,TRDMROQ128F14A597A,SOYLMYW12A6D4F7933,The Besnard Lakes,Rides The Rails
4243,6,1.000,0,0.947,TRGQFKD12903CF6A0E,SOFLCOB12AC960DD46,The Tragically Hip,Black Day In July
26558,1,0.923,0,0.922,TRTAAOP128F92D43C7,SOWESIK12AAA15D551,Monomonkey,Implode


[i] min of: key_confidence = 0.809 , mode_confidence = 0.777
[i] number of records: 240


---

### Feature Extraction
- create spectrograms of audio files
- save spectrograms for further use

Juypter Notebook <a href='./00.hlp/fft/fft.ipynb'>fft</a>

ouptuts: spectrograms (png images) of audio files with same folder structure as *src_audio* in new container path named *src_spectro*